# Business Questions on Chinook Music Store

In this tutorial we will be looking at a database called Chinook which contains information about a fictional music shop. The database includes several tables on invoice, track, album, artist etc. related to the store's sales. We will use this database to explore business questions and propositions.

The benefits of this approach are:

1. Easy to bootstrap. Analytical SQL Cell widget for Jupyter Lab is all you need.
2. Extremely efficient. Analytical SQL Cell will leverage [DuckDB](https://duckdb.org/) to query and visualize data in a performant way.
3. Free and open source. No commercial database or ChatGPT is required :-)

Chinook is a sample database available for SQL Server, Oracle, MySQL, etc. It is an alternative to the Northwind database, being ideal for demos and testing ORM tools targeting single and multiple database servers. The name of this sample database was based on the Northwind database. Chinooks are winds in the interior West of North America, where the Canadian Prairies and Great Plains meet various mountain ranges. Chinooks are most prevalent over southern Alberta in Canada. Chinook is a good name choice for a database that intents to be an alternative to Northwind.

The database includes several tables on invoice, track, album, artist etc. related to the store's sales. The Entity Relationship Diagram of the database is provided as follows:

![Chinook Database](https://m-soro.github.io/Business-Analytics/SQL-for-Data-Analysis/L4-Project-Query-Music-Store/Misc/001.png)

# Preparation

Firstly we'll need to import `asqlcell` and load extension properly.

In [ ]:
import asqlcell

%load_ext asqlcell

Now we can create a connection object to the DuckDB database file. Notice that the connection object will be used for SQL queries later.

In [ ]:
from sqlalchemy import create_engine, inspect

con = create_engine(f"duckdb:///chinook.duckdb").connect()

We can take a look at the database's schema via the connection object to understand which tables it contains:

In [ ]:
inspect(con).get_table_names()

# Exploration

## Region

Let's investigate regional information to understand the customers better.

### How many users are there per country?

We could leverage `Customer` to create a table of country name and custmer count by grouping the customers by country. We'd like to sort the countries so that the countries with most customers should appear first.

In [ ]:
%%sql --con con

SELECT
    Country,
    COUNT(CustomerId) AS Count
FROM Customer
GROUP BY 1
ORDER BY 2
DESC

We can also visualize the result set by clicking the Chart tab with the following settings:

* Chart type: Pie
* Size: Count
* Color: Country

It is obvious that most customers reside in USA, Canada and Brazil.

### How much did users spent in total per country?

We can join `Invoice` and `Customer` tables to create a result set of country name, customer name and total spent grouped by all the spendings of the customers.

In [ ]:
%%sql --con con

SELECT
    Customer.Country,
    Customer.FirstName || ' ' || Customer.LastName AS Customer,
    SUM(Invoice.Total) AS Total
FROM Invoice
JOIN Customer ON Customer.CustomerId = Invoice.CustomerId
GROUP BY 1, 2
ORDER BY 3 DESC

### How much did users spent per order?

We will need to leverage subquery refactoring or CTE to make the logic clearer. First we can create a detailed sales table via ``Invoice` and `Customer`, and then calculate the total sales as well as average order.

Even though USA and Canada users spent quite a lot of money in total, there are countries we should keep an eye on regarding money spent per order:

* Czech Republic
* India
* Germany

The sample is not big enough to risk in large budget marketing campaigns, but we should start some smaller ones to get more customers and confirm the potential market.

In [ ]:
%%sql --con con

WITH DetailedSales AS
(
    SELECT
        c.Country,
        c.CustomerId,
        i.InvoiceId,
        i.Total
    FROM Invoice i
    INNER JOIN Customer AS c ON c.CustomerId = i.CustomerId
)

SELECT
    Country,
    COUNT(DISTINCT CustomerId) AS Customers,
    SUM(Total) AS TotalSales,
    SUM(Total) / COUNT(DISTINCT InvoiceId) AS AverageOrder
FROM DetailedSales
GROUP BY Country
HAVING Customers > 1
ORDER BY TotalSales DESC, Country ASC

### Which city has the best customers?

We would like to throw a promotional Music Festival in the city we made the most money.

We could write a query that returns the cities that has the highest sum of invoice totals:

In [ ]:
%%sql --con con

SELECT
    BillingCity,
    SUM(Total) AS Total
FROM Invoice
GROUP BY 1
ORDER BY 2 DESC

# Music



In [ ]:
WITH DetailedSales AS
(
     SELECT
       c.Country AS Country,
       c.CustomerId,
       il.*
     FROM InvoiceLine il
     INNER JOIN Invoice AS i ON i.InvoiceId = il.InvoiceId
     INNER JOIN Customer AS c ON c.CustomerId = i.CustomerId
)

SELECT
    Country,
    COUNT(DISTINCT CustomerId) AS Customers,
    SUM(UnitPrice) AS TotalSales,
    SUM(UnitPrice) / COUNT(DISTINCT InvoiceId) AS AverageOrder
FROM DetailedSales
GROUP BY Country
ORDER BY TotalSales DESC, Country ASC

## Who is the best customer?

The customer who has spent the most money will be declared the best customer.

We could build a query that returns the person who has spent the most money. The solution is to link `Invoice` and `Customer` tables to retrieve this information.

In [ ]:
%%sql --con con

SELECT
    C.CustomerId,
    C.FirstName || ' ' || C.LastName AS Customer,
    SUM(I.Total) AS Total
FROM Customer C
JOIN Invoice I ON C.CustomerId = I.CustomerId
GROUP BY 1, 2
ORDER BY 3 DESC

## Who is writing the rock music?

Now that we know that our customers love rock music, we can decide which musicians to invite to play at the concert.

Let’s invite the artists who have written the most rock music in our dataset. Write a query that returns the Artist name and total track count of the top 10 rock bands. We will need to use the Genre, Track , Album, and Artist tables.

In [ ]:
%%sql --con con

SELECT
    AR.Name,
    COUNT(T.Name) AS Count
FROM Track T
JOIN Genre G ON T.GenreId = G.GenreId
JOIN Album AL ON AL.AlbumId = T.AlbumId
JOIN Artist AR ON AR.ArtistId = AL.ArtistId
WHERE G.Name = 'Rock'
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10

## How many songs base on genre does customer 12 bought?



In [ ]:
%%sql --con con

SELECT
    G.Name AS Genre,
    COUNT(T.Name)
FROM Customer C
JOIN Invoice I ON I.CustomerId = C.CustomerId
JOIN InvoiceLine IL ON IL.InvoiceId = I.InvoiceId
JOIN TRACK T ON T.TrackId = IL.TrackId
JOIN GENRE G ON G.GenreId = T.GenreId
WHERE C.CustomerId = 12
GROUP BY 1
ORDER BY 1

In [ ]:
%%sql --con con

SELECT
    FirstName,
    LastName,
    CustomerId,
    Country
FROM Customer 
WHERE Country != 'USA'

In [ ]:
%%sql --con con

SELECT
    FirstName,
    LastName,
    CustomerId,
    Country
FROM Customer 
WHERE Country == 'Brazil'

Provide a query showing the Invoices of customers who are from Brazil. The resultant table should show the customer's full name, Invoice ID, Date of the invoice and billing country.

In [ ]:
%%sql --con con

SELECT
    C.FirstName || ' ' || C.LastName AS Customer,
    I.InvoiceId,
    I.InvoiceDate,
    C.Country
FROM Customer C 
JOIN Invoice I ON C.CustomerId == I.CustomerId 
WHERE C.Country == 'Brazil'